In [1]:
import os
import pickle
import pandas as pd
import quantstats as qs
import warnings
import glob
import numpy as np

# 忽略警告
warnings.filterwarnings('ignore', category=FutureWarning)

# 定义输入和输出目录
input_dir = "reports_walkforward/"  # 收益记录文件的目录
output_dir = "reports_walkforward/top10_reports/"  # 输出HTML报告的目录

# 创建输出目录（如果不存在）
os.makedirs(output_dir, exist_ok=True)

# 存储收益数据和总收益率
results = []

# 查找所有pkl文件
pkl_files = glob.glob(os.path.join(input_dir, "*.pkl"))

print(f"找到{len(pkl_files)}个收益数据文件")

# 首先计算每个策略的总收益
for pkl_file in pkl_files:
    file_name = os.path.basename(pkl_file)
    
    # 加载pickle文件
    try:
        with open(pkl_file, 'rb') as f:
            returns = pickle.load(f)
        
        # 确保returns是正确的格式
        if isinstance(returns, pd.Series):
            # 计算总收益率 (使用累积收益)
            total_return = (1 + returns).prod() - 1
            
            # 保存结果
            results.append({
                'file_name': file_name,
                'returns': returns,
                'total_return': total_return
            })
        else:
            print(f"警告: {file_name} 不是有效的pandas Series格式，已跳过")
    except Exception as e:
        print(f"处理文件 {file_name} 时出错: {str(e)}")

# 按总收益排序并获取前10名
top_10 = sorted(results, key=lambda x: x['total_return'], reverse=True)[:10]

print("\n总收益率前10名策略:")
for i, result in enumerate(top_10, 1):
    print(f"{i}. {result['file_name']}: 总收益率 = {result['total_return']*100:.2f}%")

# 为前10名生成QuantStats报告
for i, result in enumerate(top_10, 1):
    file_name = result['file_name']
    returns = result['returns']
    total_return = result['total_return']
    
    # 从文件名提取信息 (假设命名格式为 returns_SYMBOL_timeframe.pkl)
    parts = file_name.split('_')
    if len(parts) >= 2:
        symbol = parts[1]
    else:
        symbol = "未知"
    
    # 确保返回数据格式正确
    returns = returns.copy()
    returns.index = pd.to_datetime(returns.index)
    # returns.index = returns.index.tz_convert(None)  # 移除时区信息

    # 生成报告文件名
    report_name = f"top{i}_{file_name.replace('.pkl', '')}_return{total_return*100:.2f}%.html"
    report_path = os.path.join(output_dir, report_name)
    report_title = f'前{i}名 - {symbol} - 总收益率: {total_return*100:.2f}%'
    
    # 生成QuantStats报告
    qs.reports.html(
        returns=returns,
        output=report_path,
        title=report_title,
        benchmark=None,
        strategy_name=f"MeanReverter-{symbol}"
    )
    print(f"已生成报告: {report_path}")

print("\n所有前10名报告生成完成！")

找到0个收益数据文件

总收益率前10名策略:

所有前10名报告生成完成！
